In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, auc, precision_recall_curve, log_loss, average_precision_score, mean_squared_error
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor

import pickle
from sklearn.datasets import load_boston

## 1. Prepare Data

In [ ]:
from sklearn.datasets import load_boston

np.random.seed(1234) # Scikit uses the numpy random seed state

mass = load_boston()

predictors = [var.lower() for var in mass.feature_names]
X = pd.DataFrame(mass['data'], columns=predictors)
y = pd.Series(mass['target']) # medv

X.head(), y.head()

### EAD

In [ ]:
# plot

df.hist(column='A', bins=50)
plt.show()

df.plot(x='X', y='Y', kind='scatter', title='Test', legend=False, logy=False)
plt.xlabel('D Group')
plt.ylabel('L')
plt.xlim([30, 90])
plt.ylim([0.0, 10])
plt.show()

In [ ]:
df = df.drop_duplicates(subset=['vh_id_nbr'], keep='last')

In [ ]:
# aggregate by groups
df2 = df.groupby('group2').agg({'A':np.mean, 'B':np.mean})
df2.reset_index(inplace=True)

In [ ]:
df.columns = [i if i!= 'ime' else 'response_var' for i in df.columns]

df['A'] = df['A'].apply(lambda x : 999 if x == 'UN' else x).apply(lambda x : str(int2(x)))
df['B'] = df['B'].apply(lambda x : 999 if x in [22,79,99] else x).apply(lambda x : str(int2(x)))
df['C'] = df['C'].apply(lambda x : str(int2(x)))


In [ ]:
for i in feat:
    if df[i].dtype == 'float64':
        df[i] = df[i].astype('float32')
    elif train_df[i].dtype == 'int64':
        df[i] = df[i].astype('int32')
    else:
        df[i] = df[i].astype('categorical')

In [ ]:
# change to universal column names
df.rename(columns = {'clm_nbr':'idcol'}, inplace=True)
df.rename(columns = {'response_var':'labelcol'}, inplace=True)

## 2. Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, shuffle=True)


# Reset the indexes
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

print("Train/Test size: ")
print("Train X: {} Train y: {} \nTest X: {} Test y: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

In [ ]:
# prepare data for modeling

claim_ind = 'Ind_1'
feature_names = ['A', 'B', 'S', 'T']
feat= [i for i in df.columns if i not in ['split', 'idcol', 'labelcol']]


modeling_df_x =  modeling_df[feature_names]
modeling_df_y = modeling_df[claim_ind]

train_x, test_x, train_y, test_y = train_test_split(modeling_df_x, modeling_df_y, train_size=0.7, shuffle=True)

# Reset the indexes
train_x.reset_index(inplace=True, drop=True)
train_y.reset_index(inplace=True, drop=True)
test_x.reset_index(inplace=True, drop=True)
test_y.reset_index(inplace=True, drop=True)
print("Train/Test size: ")
print("Train X: {} Train y: {} \nTest X: {} Test y: {}".format(train_x.shape, train_y.shape, test_x.shape, test_y.shape))

In [ ]:
# split into T/V
random.seed(1423)
df['split'] =  df['labelcol'].apply(lambda x: 'train' if random.random() < 0.7 else 'valid')

md_train_x = df.ix[df['split'] == 'train', :].reset_index(drop=True)
md_train_y = md_train_x.ix[:, 'labelcol']
md_valid_x = df.ix[df['split'] == 'valid', :].reset_index(drop=True)
md_valid_y = md_valid_x.ix[:, 'labelcol']
print('md_train_x.shape:', md_train_x.shape)
print('md_train_y.shape:', md_train_y.shape)
print('md_valid_x.shape:', md_valid_x.shape)
print('md_valid_y.shape:', md_valid_y.shape)

## 3. Supervised Learning

### OLS

In [ ]:
###########################################
# Model 1. Logistic Regression
###########################################

lr1 = LogisticRegression(tol=0.0001, C=0.8, penalty='l1', max_iter=1000)

# random.seed(123)
# without cv
if True:
    lr1_pred= pd.Series(lr1.fit(train_x, train_y).predict_proba(test_x)[:,1])
    top_5_accuracy(test_y, lr1_pred, 'lr1')
    print_auc(test_y, lr1_pred, 'lr1', plot=True)

def logicreg_imp_feature(logicreg_model, feat, top_n=20):
    print("Logistic Regression model Top/Bottom {} feature importance: \n".format(top_n))
    for indx in logicreg_model.coef_.argsort().ravel()[::-1][list(range(0,top_n))+list(range(-top_n,0))]:
        word = feat[indx]
        coef = logicreg_model.coef_.ravel()[indx]
        if coef != 0:
            print("{0:s}: {1:.2f}".format(word,np.exp(coef)))

            # feature importance
lr_feature_importance = logicreg_imp_feature(lr1, feat, top_n=20)

In [ ]:
def logicreg_feat_imp(logicreg_model, feature_names, top_n=10, plot=False):
    '''
    Important features in Logistic Regression
    '''
    if top_n > len(feature_names):
        top_n = len(feature_names)
    imp_df = pd.DataFrame({"feat": feature_names, "imp": np.round(logicreg_model.coef_.ravel(),3)})
    imp_df_top = imp_df.sort_values(by= ["imp"], ascending= False).iloc[:top_n, :]
    print("LogicReg model top {} feature importance:".format(top_n))
    print(imp_df_top)
    if plot:
        # bar graph to show feature importance
        pos = np.arange(imp_df_top.shape[0]) + 0.5
        plt.figure(figsize=(6, 5))
        plt.barh(pos, imp_df_top.imp.values[::-1]*100, align='center')
        plt.yticks(pos, imp_df_top.feat.values[::-1])
        plt.xlabel("Importance")
        plt.title("Feature Importance in Logistic Regression")
        plt.show()
    return imp_df_top

In [ ]:
from sklearn.linear_model import LinearRegression

ols = LinearRegression()
ols.fit(X_train, y_train)

train_score = ols.score(X_train, y_train)
test_score = ols.score(X_test, y_test)

print("R^2 on Train: {:0.2f}%, on Test: {:0.2f}%".format(train_score*100, test_score*100))

In [ ]:
from sklearn.metrics import mean_squared_error

train_score = mean_squared_error(y_train, ols.predict(X_train))
test_score = mean_squared_error(y_test, ols.predict(X_test))

print("MSE on Train: {}, on Test: {}".format(train_score, test_score))

### XGBoost

In [ ]:
####################################
# Model 2: XGBoost
####################################
# with original API

dtrain = xgb.DMatrix(train_x.values, label=train_y)
dtest = xgb.DMatrix(test_x.values, label=test_y)

evallist = [(dtest,'eval'), (dtrain,'train')]


if True:
    param = {'max_depth': 6, 'eta': 0.01, 'silent': 1, 'objective': 'binary:logistic',  
             'eval_metric':'auc', 'subsample':0.6, 'colsample_bytree':1.0} 
    num_round = 200

    xgb0 = xgb.train(params=param, dtrain=dtrain, num_boost_round=num_round,
                     evals=evallist, early_stopping_rounds=20, verbose_eval=False)

    xgb0_pred = xgb0.predict(dtest, ntree_limit=xgb0.best_iteration)
    top_5_accuracy(test_y, xgb0_pred, 'xgb0')
    print_auc(test_y, xgb0_pred, 'xgb0', plot=True)
    xgb_feature_importance = xgb_feat_imp(xgb0, feature_names, top_n=10, plot=True)
    print_precision_recall_curve(test_y, xgb0_pred, 'xgb0')



In [ ]:
def xgb_feat_imp(xgb_model, feature_names, top_n=10, plot=False):
    '''
    Important features in XGBoost
    '''
    if top_n > len(feature_names):
        top_n = len(feature_names)
    imp_df = pd.DataFrame(pd.Series(xgb_model.get_fscore(), name='imp'))
    imp_df['feat'] = imp_df.index
    imp_df['feat'] = imp_df['feat'].apply(lambda x: feature_names[int(x[1:])])
    imp_df.reset_index(drop=True, inplace=True)
    imp_df_top = imp_df.sort_values(by='imp', ascending=False).iloc[:top_n, :]
    imp_df_top['imp'] = np.round(imp_df_top['imp'] / imp_df['imp'].sum(), 3)
    imp_df_top = imp_df_top[['feat', 'imp']]
    print('XGBoost model top {} feature importance:'.format(top_n))
    print(imp_df_top)
    if plot:
        # bar graph to show feature importance
        pos = np.arange(imp_df_top.shape[0]) + 0.5
        plt.figure(figsize=(6, 5))
        plt.barh(pos, imp_df_top.imp.values[::-1]*100, align='center')
        plt.yticks(pos, imp_df_top.feat.values[::-1])
        plt.xlabel("Importance")
        plt.title("Feature Importance in XGBoost")
        plt.show()
    return imp_df_top

In [ ]:
def fit_xgb(df, response='ime'):
    gc.collect()
    # response need to be binary
    np.random.seed(123456)
    msk = np.random.rand(len(df)) < 0.8
    md_train_x = df[msk].loc[:, df.columns != response].reset_index(drop=True)
    md_train_y = df[msk].loc[:, df.columns == response].reset_index(drop=True)
    md_valid_x = df[~msk].loc[:, df.columns != response].reset_index(drop=True)
    md_valid_y = df[~msk].loc[:, df.columns == response].reset_index(drop=True)
    dtrain = xgb.DMatrix(md_train_x.values, label=md_train_y.values, feature_names=md_train_x.columns)
    dvalid = xgb.DMatrix(md_valid_x.values, label=md_valid_y.values, feature_names=md_valid_x.columns)
    gc.collect()


    param = {'max_depth': 6,
            'eta': 0.1,
            'silent': 1,
            'objective': 'binary:logistic',
            'colsample_bytree' : 0.7,
            'min_child_weight ': 10,
            'nthread': 8,
            # 'eval_metric': 'auc',
            'eval_metric': 'error',
            "eta_decay": 0.5,
            'scale_pos_weight': 5,
            }

    num_round = 600

    evallist = [(dvalid,'eval'), (dtrain,'train')]
    xgb_model = xgb.train(params=param, dtrain=dtrain, num_boost_round=num_round, evals=evallist, early_stopping_rounds=20, verbose_eval=False)

    xgb_pred = xgb_model.predict(dvalid, ntree_limit=xgb_model.best_iteration)
    xgb_pred_train = xgb_model.predict(dtrain, ntree_limit=xgb_model.best_iteration)

    # xgb_obj = "../xgb_obs_" + str(obs) + "features_"+ str(nf) + ".p"
    # pickle.dump(xgb_model, open(xgb_obj, "wb"))

    result = [xgb_model, top_5_accuracy(md_valid_y, xgb_pred, 'xgb valid') + '\n' + top_5_accuracy(md_train_y, xgb_pred_train, 'xgb train')]
    return result

def plot_xgb_imp(xgb_model, path, txt='', fname='abc'):
    with PdfPages(path) as pdf:
        f, ax = plt.subplots(figsize=(12,50))
        importances = xgb_model.get_fscore()
        importance_frame = pd.DataFrame({'Importance': list(importances.values()), 'Feature': list(importances.keys())})
        importance_frame.sort_values(by = 'Importance', inplace = True, ascending=False)
        importance_frame.to_csv("var_importance_" + fname + ".csv", index=False)

        importance_frame[:150].sort_values(by = 'Importance', ascending=True).plot(kind = 'barh', x = 'Feature', color = 'orange', ax=ax, grid=True)

        ax.text(0.95, 0.04, txt,
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='blue', fontsize=15)
        plt.tight_layout()
        pdf.savefig()
        plt.close()


In [ ]:
# with sklearn wrapper

from xgboost import XGBRegressor

bst = XGBRegressor(learning_rate=0.1
                    , max_depth=3
                    , min_child_weight=20
                    , subsample=0.9
                    , n_estimators=1000)

bst.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

train_score = mean_squared_error(y_train, bst.predict(X_train, ntree_limit=bst.best_iteration))
test_score = mean_squared_error(y_test, bst.predict(X_test, ntree_limit=bst.best_iteration))

print("MSE on Train: {}, on Test: {}".format(train_score, test_score))

In [ ]:
# with sklearn API

xgb1 = xgb.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=600, silent=False,
                         objective='binary:logistic', nthread=-1, subsample=0.6, colsample_bytree=0.7)
if True:
    xgb1_pred = pd.Series(xgb1.fit(md_train_x[feat], md_train_y, early_stopping_rounds=20, eval_metric="error", 
                                   eval_set=[(md_valid_x[feat], md_valid_y)], 
                                   verbose=False).predict_proba(md_valid_x[feat])[:,1])
    top_5_accuracy(md_valid_y, xgb1_pred, 'xgb1')
    print_auc(md_valid_y, xgb1_pred, 'xgb1')

if False:
    xgb1_train_cv, xgb1_pred_cv, xgb1_cv = cvModel(md_train_x, md_valid_x, target= md_train_y, model= xgb1, feat= feat,
                                          idcol= "idcol", nfolds= 4, nreps= 2, classify= True)
    top_5_accuracy(xgb1_pred_cv.target.values, xgb1_pred_cv.pred.values, 'xgb1_cv_train')
    print_auc(xgb1_pred_cv.target.values, xgb1_pred_cv.pred.values, 'xgb1_cv_train')
    top_5_accuracy(md_valid_y, xgb1_pred_cv.pred.values, 'xgb1_cv_pred')
    print_auc(md_valid_y, xgb1_pred_cv.pred.values, 'xgb1_cv_pred')

# xgb1  top 5% accuracy:  0.47004
# xgb1  AUC:  0.89939

### GBM

In [ ]:
#######################################################
# Model 3. GBM
#######################################################


if False:
    gbm1 = GradientBoostingClassifier(loss = "deviance", learning_rate= 0.05, n_estimators= 100,
                                      max_depth=1, min_samples_split= 10, min_samples_leaf= 10,
                                      subsample= 0.5, max_features= None, verbose= 0)
    gbm1_pred = pd.Series(gbm1.fit(train_x, train_y).predict_proba(test_x)[:,1])
    top_5_accuracy(test_y, gbm1_pred, 'gbm1')
    print_auc(test_y, gbm1_pred, 'gbm1', plot=True)
    gbm_feature_importance = gbm_feat_imp(gbm1, feature_names, top_n=10, plot=True)
    print_precision_recall_curve(test_y, gbm1_pred, 'gbm1')
    

if True:
    gbm2 = GradientBoostingRegressor(loss = "huber", learning_rate= 0.05, n_estimators= 100,
                                     max_depth=4, min_samples_split= 10, min_samples_leaf= 10,
                                     subsample= 0.5, max_features= None, verbose= 0)
    gbm2_pred = pd.Series(gbm2.fit(train_x, train_y).predict(test_x))
    mse = mean_squared_error(test_y, gbm2_pred)
    print("MSE: {0:.5f} from gbm2".format(mse))
    gbm_feature_importance = gbm_feat_imp(gbm2, feature_names, top_n=10, plot=True)

# important variables in gbm
top_n = 20
gbm_feat_imp = pd.DataFrame({"feat": feat, "imp": np.round(gbm1.feature_importances_,3)})
gbm_feat_imp_top = gbm_feat_imp.sort_values(by= ["imp"], ascending= False).iloc[:top_n, :]
print(gbm_feat_imp_top)
# bar graph to show feature importance
gbm_feat_imp_bar = gbm_feat_imp.sort_values(by= ["imp"], ascending= True).iloc[-top_n:, :]
pos = np.arange(gbm_feat_imp_bar.shape[0]) + 0.5
plt.figure(figsize=(12, 10))
plt.subplot(1, 1, 1)
plt.barh(pos, gbm_feat_imp_bar.imp.values*100, align='center')
plt.yticks(pos, gbm_feat_imp_bar.feat.values)
plt.xlabel("Importance")
plt.title("Variable Importance in Gradient Boosting")
plt.show()

In [ ]:
def gbm_feat_imp(gbm_model, feature_names, top_n=10, plot=False):
    '''
    Important features in GBM
    '''
    if top_n > len(feature_names):
        top_n = len(feature_names)
    imp_df = pd.DataFrame({"feat": feature_names, "imp": np.round(gbm_model.feature_importances_,3)})
    imp_df_top = imp_df.sort_values(by= ["imp"], ascending= False).iloc[:top_n, :]
    print('GBM model top {} feature importance:'.format(top_n))
    print(imp_df_top)
    if plot:
        # bar graph to show feature importance
        pos = np.arange(imp_df_top.shape[0]) + 0.5
        plt.figure(figsize=(6, 5))
        plt.barh(pos, imp_df_top.imp.values[::-1]*100, align='center')
        plt.yticks(pos, imp_df_top.feat.values[::-1])
        plt.xlabel("Importance")
        plt.title("Feature Importance in GBM")
        plt.show()
    return imp_df_top

### Random Forest

In [ ]:
#######################################################
# Model 4. Random Forests
#######################################################

rf1 = RandomForestClassifier(n_estimators=1000, min_samples_split=10, criterion='entropy', random_state=123, 
                             n_jobs=-1, verbose=0)

if True:
    rf1_pred = pd.Series(rf1.fit(train_x, train_y).predict_proba(test_x)[:,1])
    top_5_accuracy(test_y, rf1_pred, 'rf1')
    print_auc(test_y, rf1_pred, 'rf1', plot=True)
    
if False:
    rf1_train_cv, rf1_pred_cv, rf1_cv = cvModel(md_train_x, md_valid_x, target=md_train_y, model=rf1, feat=feat,
                                          idcol="idcol", nfolds=10, nreps=2, classify=True)
    top_5_accuracy(md_valid_y, rf1_pred_cv.pred.values, 'rf1_cv')
    print_auc(md_valid_y, rf1_pred_cv.pred.values, 'rf1_cv')

# feature importance
rf_feature_importance = rf_feat_imp(rf1, feature_names, top_n=10, plot=True)


# feature importance
rf_feature_importance = pd.DataFrame(list(zip(md_train_x[feat], rf1_cv.feature_importances_)))
rf_feature_importance.columns = ['feature', 'importance']
rf_feature_importance.sort_values(by='importance', ascending=False).head(20)

In [ ]:
def rf_feat_imp(rf_model, feature_names, top_n=10, plot=False):
    '''
    Important features in Random Forest
    '''
    if top_n > len(feature_names):
        top_n = len(feature_names)
    imp_df = pd.DataFrame({"feat": feature_names, "imp": np.round(rf_model.feature_importances_,3)})
    imp_df_top = imp_df.sort_values(by= ["imp"], ascending= False).iloc[:top_n, :]
    print('Random Forest top {} feature importance:'.format(top_n))
    print(imp_df_top)
    if plot:
        # bar graph to show feature importance
        pos = np.arange(imp_df_top.shape[0]) + 0.5
        plt.figure(figsize=(6, 5))
        plt.barh(pos, imp_df_top.imp.values[::-1]*100, align='center')
        plt.yticks(pos, imp_df_top.feat.values[::-1])
        plt.xlabel("Importance")
        plt.title("Feature Importance in Random Forest")
        plt.show()
    return imp_df_top



In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500
                           , max_features="sqrt"
                           , max_depth=5
                           , min_samples_split=30
                           , oob_score=True
                            )
rf.fit(X_train, y_train)


train_score = mean_squared_error(y_train, rf.predict(X_train))
test_score = mean_squared_error(y_test, rf.predict(X_test))

print("MSE on Train: {}, on Test: {}".format(train_score, test_score))

### ERT

In [ ]:
#######################################################
# Model 5. ERT
#######################################################

ert1 = ExtraTreesClassifier(n_estimators=1000, min_samples_split=10, criterion='entropy', random_state=1234, 
                            n_jobs=-1, verbose=0)

if True:
    ert1_pred = pd.Series(ert1.fit(md_train_x[feat], md_train_y).predict_proba(md_valid_x[feat])[:,1])
    top_5_accuracy(md_valid_y, ert1_pred, 'ert1')
    print_auc(md_valid_y, ert1_pred, 'ert1')

if False:
    ert1_train_cv, ert1_pred_cv, ert1_cv = cvModel(md_train_x, md_valid_x, target=md_train_y, model=ert1, feat=feat,
                                          idcol="idcol", nfolds=4, nreps=3, classify=True)
    top_5_accuracy(md_valid_y, ert1_pred_cv.pred.values, 'ert1_cv')
    print_auc(md_valid_y, ert1_pred_cv.pred.values, 'ert1_cv')

### LDA topic modeling

In [ ]:
###########################################
# Model 6. LDA topic modeling
###########################################

# LDA topic
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,  learning_method='online',
                                learning_offset=50., random_state=123, n_jobs=6, verbose=0)
lda.fit(tf[feat])
#key words
# n_top_words = 20
# LDA_top_words(lda, tf_vectorizer, n_top_words)

# logistic regression on topics
lda_train_x = normalize(lda.transform(md_train_x[feat]), norm='l1', axis=1)
lda_valid_x = normalize(lda.transform(md_valid_x[feat]), norm='l1', axis=1)
lda_topic_lr = LogisticRegression(C=0.5, penalty='l1')
lda_topic_lr_pred = pd.Series(lda_topic_lr.fit(lda_train_x, md_train_y).predict_proba(lda_valid_x)[:,1])

print("{} non-zero features".format(np.sum(lda_topic_lr.coef_!=0)))
# score to get top 5% accuracy and AUC
top_5_accuracy(md_valid_y, lda_topic_lr_pred, 'lda_topic_lr')
print_auc(md_valid_y, lda_topic_lr_pred, 'lda_topic_lr')

# topic importance
print("topic importance:")
for indx in lda_topic_lr.coef_.argsort().ravel()[::-1]:
    coef = lda_topic_lr.coef_.ravel()[indx]
    if coef != 0:
        print("Topic {0:n}: {1:.2f}".format(indx,np.exp(coef)))

### SGD

In [ ]:
#######################################################
# Model 7. SGD
#######################################################

sgd1 = SGDClassifier(loss="modified_huber", penalty="l1", n_iter=100, random_state=1234, n_jobs=6)
sgd1.fit(md_train_x[feat], md_train_y)

sgd1_pred = pd.Series(sgd1.predict_proba(md_valid_x[feat])[:,1])

# score to get top 5% accuracy
top_5_accuracy(md_valid_y, sgd1_pred, 'sgd1')
print_auc(md_valid_y, sgd1_pred, 'sgd1')

### Cross Validation

In [ ]:
from sklearn.model_selection import KFold

def kf(X=X_train):
    kf = KFold(n_splits=5, random_state=1234)
    return kf.split(X) # we can plug this generator into later functions to run CV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Initialize a Random Forest
rf_cv = RandomForestRegressor()

# Set up the list of parameters to grid over
param_grid = {
        'n_estimators': [100, 500, 750, 1000],
        'max_features': ["sqrt", 0.33],
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [2, 15, 30, 50]
}

# Using the empty RF as a base, pass the parameters into a GridSearchCV
cv_grid = GridSearchCV(rf_cv, param_grid, cv=kf(), n_jobs=12) # n_folds=12 adds multithreading with 12 cores
cv_grid.fit(X_train, y_train)

# GridSearchCV .predict() uses whichever parameters gave the best mean OOF score
train_score = mean_squared_error(y_train, cv_grid.predict(X_train))
test_score = mean_squared_error(y_test, cv_grid.predict(X_test))

print("MSE on Train: {}, on Test: {}".format(train_score, test_score))

print("Best Parameters: {}".format(cv_grid.best_params_))

## 4. Ensemble

In [ ]:
# random search of weights for ensembling

models = pd.DataFrame([lr1_pred, xgb1_pred, gbm1_pred, rf1_pred, ert1_pred])
# models = pd.DataFrame([lr1_pred_cv.pred, xgb1_pred_cv.pred, gbm1_pred_cv.pred, rf1_pred_cv.pred, ert1_pred_cv.pred])
models = models.T
models.columns = ['lr1_pred', 'xgb1_pred', 'gbm1_pred', 'rf1_pred', 'ert1_pred']
weights, _ = random_search_ensemble_weights(models, target=md_valid_y, n_random_runs=100000, random_state=123456)

In [ ]:

def random_search_ensemble_weights(models, target, n_random_runs=100, random_state=123456):
    n_models = models.shape[1]
    print('Number of models to ensemble: {}'.format(n_models))
    random.seed(random_state)
    accuracy_cup = []
    w = []
    for i in range(n_random_runs):
        w = [random.uniform(0.0, 1.0) for j in range(n_models)]
        w = np.array(w) / sum(w)
        models['pred_combined'] = models.iloc[:, 0] * w[0]
        for m in range(1, n_models):
            models['pred_combined'] = models['pred_combined'] + models.iloc[:, m] * w[m]

        actual_pred = pd.concat([pd.DataFrame(target), models['pred_combined']], axis=1)
        actual_pred.columns = ['actual','pred']
        actual_pred = actual_pred.sort_values(by='pred', ascending=0)
        top_5_percent_cutoff = int(len(actual_pred)*0.05)
        accuracy = sum(actual_pred['actual'][:top_5_percent_cutoff])/top_5_percent_cutoff
        accuracy_cup.append([accuracy, str(w)])

    accuracy_cup =  pd.DataFrame(accuracy_cup)
    accuracy_cup.columns = ['accuracy', 'weights']
    max_accuracy = accuracy_cup[accuracy_cup.accuracy == accuracy_cup.accuracy.max()].values[0][0]
    best_weights = accuracy_cup[accuracy_cup.accuracy == accuracy_cup.accuracy.max()].values[0][1]
    print('random.seed:', random_state, 'n_random_runs:', n_random_runs)
    print('Max accuracy: ', max_accuracy)
    print('Weights: ', best_weights)
    return best_weights, max_accuracy


In [ ]:
# ensemble 

pred_combined = (lr1_pred * 0.23161068   + xgb1_pred * 0.35029858 + 
                 gbm1_pred * 0.05933682 + rf1_pred * 0.07465767 + ert1_pred * 0.28409625)

top_5_accuracy(md_valid_y, pred_combined, 'ensembled_models')
print_auc(md_valid_y, pred_combined, 'ensembled_models')

## 5. Performance Metrics

In [ ]:
def top_5_accuracy(actual, pred, model_name_to_print):
    actual_pred = pd.concat([pd.DataFrame(actual), pd.DataFrame(pred)], axis=1)
    actual_pred.columns = ['actual','pred']
    actual_pred = actual_pred.sort_values(by='pred', ascending=0)
    top_5_percent_cutoff = int(len(actual_pred)*0.05)
    accuracy = sum(actual_pred['actual'][:top_5_percent_cutoff])/top_5_percent_cutoff
    result = str(model_name_to_print) + ' top 5% accuracy: ' + str(np.round(accuracy, 5))
    return result

def top_5_accuracy(actual, pred, model_name_to_print):
    actual_pred = pd.concat([pd.DataFrame(actual), pd.DataFrame(pred)], axis=1)
    actual_pred.columns = ['actual','pred']
    actual_pred = actual_pred.sort_values(by='pred', ascending=0)
    top_5_percent_cutoff = int(len(actual_pred)*0.05)
    accuracy = sum(actual_pred['actual'][:top_5_percent_cutoff])/top_5_percent_cutoff
    print(str(model_name_to_print), ' top 5% accuracy: ', np.round(accuracy, 5))
    return np.round(accuracy, 5)

def print_auc(actual, pred, model_name_to_print, plot=False):
    fpr, tpr, thresholds = roc_curve(actual, pred)
    roc_auc = auc(fpr, tpr)
    if plot:
        plt.plot(fpr, tpr, label=model_name_to_print)
        plt.legend(loc='best')
        plt.title(('ROC of ' + model_name_to_print + ', AUC = '+ str(np.round(roc_auc, 5))))
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.ylim([0.0, 1.0])
        plt.xlim([0.0, 1.0])
        plt.show()
    print(str(model_name_to_print), ' AUC: ', np.round(roc_auc, 5))
    return roc_auc

def print_precision_recall_curve(actual, pred, model_name_to_print):
    precision, recall, threshold = precision_recall_curve(actual, pred)
    average_precision = average_precision_score(actual, pred)
    plt.plot(recall, precision, color='b', alpha=1, label=model_name_to_print)
    plt.legend(loc='best')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.0])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve: Average Precision={0:0.4f}'.format(
        average_precision))
    plt.show()

## 6. Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('imputer', Imputer(missing_values='NaN', strategy='mean')),
    ('scaler', StandardScaler(with_mean=True, with_std=True)),
    ('rf', RandomForestRegressor()),
])

# Parameters can be passed anywhere in the pipeline, so they begin with the step name
param_grid = {
        'rf__n_estimators': [100, 500, 750, 1000],
        'rf__max_features': ["sqrt", 0.33],
        'rf__max_depth': [3, 5, 7, 10],
        'rf__min_samples_split': [2, 15, 30, 50]
}

# Create the CV Grid Search with the pipeline as the model
pipeline_grid = GridSearchCV(pipeline, param_grid, cv=kf(), n_jobs=12)
pipeline_grid.fit(X_train, y_train)

# Predict
train_score = mean_squared_error(y_train, pipeline_grid.predict(X_train))
test_score = mean_squared_error(y_test, pipeline_grid.predict(X_test))

print("MSE on Train: {}, on Test: {}".format(train_score, test_score))